# Simple demo of contexts, monitors, and dependencies

The basic class is just a function from X to Y.
The class provides some numerical derivatives:
* delta() - dY/dX
* gamma() - dDelta/dX


1. **Contexts** are used to provide values when X has been shifted by some epsilon amount. In earlier demos, these were
only used to hold timestamps for database views, but they are quite general.
1. **Monitors** provide some logging/summarization of what is being done under the covers.
1. **Dependencies** are the map from computations to their inputs.  

Note that there is some caching behind the scenes now that the system understands dependencies.

In [1]:
import mand.core

from mand.core import Entity, node, Context
from mand.core import PrintMonitor, SummaryMonitor

## A simple function [User]

This class just computes Y = X*X*X.

Note the greeks (derivatives) are singled-sided, so this has some bias/inaccuracy, but it's simplier to understand.

In [2]:
class O(Entity):

    @node
    def X(self):
        return 10
    
    @node
    def Y(self):
        return self.X()**3
    
    @node
    def eps(self):
        return 1e-4
    
    @node 
    def delta(self):
        # bad: single-sided delta
        eps = self.eps()
        y = self.Y()
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump}, name='delta-up')
        with c:
            y_bump = self.Y()
        return (y_bump-y)/eps
    
    @node 
    def gamma(self):
        eps = self.eps()
        d = self.delta()
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump}, name='gamma-up')
        with c:
            d_bump = self.delta()
        return (d_bump-d)/eps

In [3]:
with PrintMonitor():
    o = O()
    d = o.delta()
    print 'delta:', d

 GetValue begin ctx: Root, key: O@108357290/O:delta
   GetValue/Calc begin ctx: Root, key: O@108357290/O:delta
     GetValue begin ctx: Root, key: O@108357290/O:eps
       GetValue/Calc begin ctx: Root, key: O@108357290/O:eps
         GetValue/Calc end ctx: Root, key: O@108357290/O:eps
       GetValue end value: 0.0001, ctx: Root, key: O@108357290/O:eps
     GetValue begin ctx: Root, key: O@108357290/O:Y
       GetValue/Calc begin ctx: Root, key: O@108357290/O:Y
         GetValue begin ctx: Root, key: O@108357290/O:X
           GetValue/Calc begin ctx: Root, key: O@108357290/O:X
             GetValue/Calc end ctx: Root, key: O@108357290/O:X
           GetValue end value: 10, ctx: Root, key: O@108357290/O:X
         GetValue/Calc end ctx: Root, key: O@108357290/O:Y
       GetValue end value: 1000, ctx: Root, key: O@108357290/O:Y
     GetValue begin ctx: Root, key: O@108357290/O:X
       GetValue from ctx value: 10, ctx: Root, key: O@108357290/O:X
     GetValue begin ctx: Root, key: O@10

In [4]:
d = o.delta()
print 'delta:', d
g = o.gamma()
print 'gamma:', g

delta: 300.003000009
gamma: 60.0006046625


## Showing captured dependencies

Note the nested context for delta inside the gamma context.

In [5]:
c = Context.current()
cbm = o.gamma
n = c.getCBM(cbm)
n.printInputGraph()

 <O@108357290/O:gamma in Root>
   <O@108357290/O:eps in Root>
   <O@108357290/O:delta in Root:gamma-up>
     <O@108357290/O:eps in Root:gamma-up>
     <O@108357290/O:Y in Root:gamma-up:delta-up>
       <O@108357290/O:X in Root:gamma-up:delta-up>
     <O@108357290/O:X in Root:gamma-up>
     <O@108357290/O:Y in Root:gamma-up>
       <O@108357290/O:X in Root:gamma-up>
   <O@108357290/O:delta in Root>
     <O@108357290/O:eps in Root>
     <O@108357290/O:Y in Root:delta-up>
       <O@108357290/O:X in Root:delta-up>
     <O@108357290/O:X in Root>
     <O@108357290/O:Y in Root>
       <O@108357290/O:X in Root>
   <O@108357290/O:X in Root>


## O2 - the two-sided derivative version of O. [User]

More version of the O class: we now bump X up and down, taking the average of the dependent variable changes.

In [6]:
class O2(Entity):

    @node
    def X(self):
        return 10
    
    @node
    def Y(self):
        return self.X()**3
    
    @node
    def eps(self):
        return 1e-4
    
    @node
    def upCtx(self):
        x_bump = self.X() + self.eps()
        c = Context({self.X: x_bump}, name='X-up')
        return c
    
    @node
    def downCtx(self):
        x_bump = self.X() - self.eps()
        c = Context({self.X: x_bump}, name='X-down')
        return c
    
    @node 
    def delta(self):
        eps = self.eps()
        with self.upCtx():
            y_up = self.Y()
        with self.downCtx():
            y_down = self.Y()
        return (y_up-y_down)/eps/2
    
    @node 
    def gamma(self):
        eps = self.eps()
        with self.upCtx():
            delta_up = self.delta()
        with self.downCtx():
            delta_down = self.delta()
        return (delta_up-delta_down)/eps/2
    
with PrintMonitor():
    o2 = O2()
    d = o2.delta()
    print 'delta:', d
    g = o2.gamma()
    print 'gamma:', g

 GetValue begin ctx: Root, key: O2@10837c110/O2:delta
   GetValue/Calc begin ctx: Root, key: O2@10837c110/O2:delta
     GetValue begin ctx: Root, key: O2@10837c110/O2:eps
       GetValue/Calc begin ctx: Root, key: O2@10837c110/O2:eps
         GetValue/Calc end ctx: Root, key: O2@10837c110/O2:eps
       GetValue end value: 0.0001, ctx: Root, key: O2@10837c110/O2:eps
     GetValue begin ctx: Root, key: O2@10837c110/O2:upCtx
       GetValue/Calc begin ctx: Root, key: O2@10837c110/O2:upCtx
         GetValue begin ctx: Root, key: O2@10837c110/O2:X
           GetValue/Calc begin ctx: Root, key: O2@10837c110/O2:X
             GetValue/Calc end ctx: Root, key: O2@10837c110/O2:X
           GetValue end value: 10, ctx: Root, key: O2@10837c110/O2:X
         GetValue begin ctx: Root, key: O2@10837c110/O2:eps
           GetValue from ctx value: 0.0001, ctx: Root, key: O2@10837c110/O2:eps
         Context create ctx: Root:X-up
         Context set value: 10.0001, ctx: Root:X-up, key: O2@10837c110/O2

## Summary Monitor [Test]

This monitor just prints basic summary information about a computation.

Note that 6 contexts were created: 2 for the gamma up/down, and then two nested contexts for delta within those gamma
contexts.

In [7]:
with SummaryMonitor():
    o2a = O2()
    d = o2a.delta()
    print 'delta:', d
    g = o2a.gamma()
    print 'gamma:', g

delta: 300.000000009
gamma: 59.9999992801
Compute activity:
               Context:     6
              GetValue:    40
         GetValue/Calc:    20
